# Projeto Final - ***Natural Language Processing***


João Victor de Almeida Braga

Sidney Barbosa de Oliveira

Diogo Gindler Diniz



**Introdução**

Muitos modelos de alto desempenho exigem que todos os dados, incluindo os de teste, sejam "vistos" durante o treinamento (aprendizado transdutivo). Essa abordagem, embora eficaz em benchmarks, é inviável para aplicações do mundo real que exigem a classificação de dados novos e nunca vistos. A solução prática e necessária para a generalização é o aprendizado indutivo.

**Objetivo**

O objetivo central do projeto é investigar a viabilidade de um modelo híbrido BERT-GNN indutivo. Os treinos do modelo são feitos apenas com dados conhecidos, aprendendo a generalizar de forma robusta. Assim, pretendemos verificar se o modelo consegue manter o alto desempenho dos baselines transdutivos, ao mesmo tempo que adquire flexibilidade e aplicabilidade prática de um modelo indutivo. 

In [ ]:
# imports do pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset 
from transformers import BertTokenizer, BertModel, logging

# Desliga os warnings do Hugging Face 
logging.set_verbosity_error() 

# PyTorch Geometric
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool

# NLTK 
import nltk
from nltk.corpus import stopwords

# Scikit-learn (Métricas) 
from sklearn.metrics import accuracy_score

# Verificação de Hardware (CRÍTICO)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"Nome da GPU: {torch.cuda.get_device_name(0)}")
print("---------------------------------")

In [ ]:
# Stopwords do NLTK 
nltk.download('stopwords')
stop_words_set = set(stopwords.words('english'))
print(f"Carregadas {len(stop_words_set)} stopwords do NLTK.")

# Constantes do Projeto
BERT_MODEL_NAME = 'bert-base-uncased'
DATA_DIR = "./data" 

# Hiperparâmetros de Treinamento 
LEARNING_RATE = 2e-5 
BATCH_SIZE = 2 
NUM_EPOCHS = 4  
HIDDEN_DIM = 256 
MAX_LENGTH = 200 

### Preparação Inicial do DataSet e Definição de Estruturas

***Nessa Etapa:***

Preparamos o dataset IMDB para um modelo de Rede Neural Gráfica (GNN). Primeiramente, ele carrega e inspeciona os documentos de treino e teste, definindo estruturas de dados com a classe TextGraphDataset para encapsular textos e rótulos. Em seguida, inicializa o BertTokenizer para processamento textual. 

O build_token_graph implementa a lógica de conversão de sequências de tokens em grafos, ou seja, ele estabelece arestas entre tokens adjacentes dentro de uma janela de co-ocorrência, fundamental para modelar a relação de proximidade entre palavras e permitir a propagação de mensagens do GNN

In [ ]:
from datasets import load_dataset

# Carregando DataSet do IMDB
print("Baixando e carregando dataset IMDB...")
imdb_dataset = load_dataset("imdb")

train_texts_cleaned = imdb_dataset['train']['text']
train_labels = imdb_dataset['train']['label']

test_texts_cleaned = imdb_dataset['test']['text']
test_labels = imdb_dataset['test']['label']

# Definindo classes
num_classes = 8 

print(f"Total de classes: {num_classes}")
print(f"Documentos de treino: {len(train_texts_cleaned)}")
print(f"Documentos de teste: {len(test_texts_cleaned)}")

# Classe TextGraphDataset 
class TextGraphDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Instâncias do Dataset 
train_dataset = TextGraphDataset(train_texts_cleaned, train_labels)
test_dataset = TextGraphDataset(test_texts_cleaned, test_labels)

# Tokenizador 
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

def build_token_graph(num_tokens, window_size=2):
    edge_index_list = []
    for i in range(num_tokens):
        for j in range(max(0, i - window_size), min(num_tokens, i + window_size)):
            if i != j:
                edge_index_list.append([i, j])
                
    if not edge_index_list:
        return torch.tensor([[0], [0]], dtype=torch.long)
    
    return torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()

### Preparação e Limpeza do Dataset R8

***Nessa Etapa:***

Este código implementa o pré-processamento do dataset R8: ele carrega os dados, executa uma limpeza no texto: removendo pontuações, números, stopwords e palavras curtas. Em seguida, mapeia os rótulos textuais únicos para índices numéricos sequenciais, preparando os conjuntos de treino e teste para um modelo de classificação com $N$ classes (onde $N$ é o total de rótulos únicos encontrados).

In [ ]:
def load_data_and_labels(filepath):
    """Lê os arquivos de dados do R8"""
    texts = []
    labels = []
    with open(filepath, 'r', encoding='latin-1') as f:
        for line in f:
            line = line.strip()
            if not line: continue
            parts = line.split('\t', 1)
            if len(parts) == 2:
                label, text = parts
                texts.append(text)
                labels.append(label)
    return texts, labels

def clean_text(text):
    """Limpa o texto"""
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = text.lower()
    words = text.split()
    cleaned_words = [w for w in words if w not in stop_words_set and len(w) > 2]
    return " ".join(cleaned_words)

# Carregamento de Dados
train_texts_raw, train_labels_str = load_data_and_labels(os.path.join(DATA_DIR, 'train.txt'))
test_texts_raw, test_labels_str = load_data_and_labels(os.path.join(DATA_DIR, 'test.txt'))

train_texts_cleaned = [clean_text(text) for text in train_texts_raw]
test_texts_cleaned = [clean_text(text) for text in test_texts_raw]

# Mapeamento de Labels
labels_unique = sorted(list(set(train_labels_str)))
num_classes = len(labels_unique)
label_map = {label: i for i, label in enumerate(labels_unique)}

train_labels = [label_map[label] for label in train_labels_str]
test_labels = [label_map[label] for label in test_labels_str]

print(f"Total de classes: {num_classes} ({labels_unique[0]}, {labels_unique[1]}, ...)")
print(f"Documentos de treino: {len(train_texts_cleaned)}")
print(f"Documentos de teste: {len(test_texts_cleaned)}")

### Configuração De Batch Híbrido

***Nessa Etapa:***

Esta função, collate_fn_bert_gnn, é uma rotina de agrupamento customizada crucial para modelos híbridos que combinam BERT e Redes Neurais Gráficas (GNN). Ela recebe um batch de textos e rótulos e realiza o processamento duplo: primeiro, tokeniza e padroniza os textos para criar os inputs do BERT (bert_inputs), e em seguida, para cada item, usa a máscara de atenção do BERT para determinar o número real de tokens e constrói o edge_index (a estrutura de adjacência de grafo) usando a função build_token_graph. 

In [ ]:
def collate_fn_bert_gnn(batch):

    texts, labels = zip(*batch)
    
    # Tokeniza o lote de textos com padding
    bert_inputs = tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors="pt"
    )
    
    # Processa para a GNN
    data_list = []
    for i in range(len(texts)):
        # Pega o número real de tokens 
        num_tokens = int(bert_inputs['attention_mask'][i].sum())
        
        # Constrói o grafo de tokens para este item
        edge_index = build_token_graph(num_tokens)
        
        # O y (label) para este item
        y = torch.tensor(labels[i], dtype=torch.long)
        
        # Criamos um Data object sem x. O x virá do BERT
        data_list.append(Data(edge_index=edge_index, y=y))
    
    # Empacota a GNN
    pyg_batch = Batch.from_data_list(data_list)
    
    # Retorna tudo
    return {
        'bert_inputs': bert_inputs, 
        'pyg_batch': pyg_batch       
    }

### Inicialização dos DataLoaders

***Nessa Etapa:***

Criamos os DataLoaders de treino e teste do PyTorch. A configuração chave é usar a função collate_fn_bert_gnn para garantir que os dados sejam agrupados corretamente para o modelo híbrido BERT-GNN. O loader de treino embaralha os dados, e o de teste não. O código finaliza exibindo o número de lotes (batches) disponíveis em cada conjunto

In [ ]:
# Usa o DataLoader padrão do PyTorch, mas com nosso collate_fn
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn_bert_gnn,
    num_workers=0,
    pin_memory=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn_bert_gnn,
    num_workers=0,
    pin_memory=False
)

print(f"Total de lotes de treino: {len(train_loader)}")
print(f"Total de lotes de teste: {len(test_loader)}")

### Modelo Híbrido BERT-GNN para Fine-Tuning 

***Nessa Etapa:***

Definimos e inicializamos a arquitetura HybridGNN_FineTune, um modelo que integra o BERT para extrair embeddings contextuais de cada token (no modo fine-tuning) e duas camadas GCN para modelar as relações entre tokens. No método forward, o modelo filtra o padding do output do BERT para obter features de nó, processa-as pela GNN, utiliza o pooling para condensar a informação do grafo em um vetor de documento, e finaliza com uma camada linear para a classificação.

Por fim, o código instancia o modelo e define a função de perda e o otimizador, preparando a estrutura para o treinamento

In [ ]:
class HybridGNN_FineTune(nn.Module):
    def __init__(self, hidden_dim, output_dim, dropout_prob=0.5):
        super(HybridGNN_FineTune, self).__init__()
        
        # Camada BERT (Será treinada)
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME)
        bert_output_dim = self.bert.config.hidden_size # 768
        
        # Camadas GNN
        self.gcn1 = GCNConv(bert_output_dim, hidden_dim)
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        
        # Pooling (Agrega os nós de tokens em um vetor de documento)
        self.pool = global_mean_pool
        
        # Camada de Classificação Final
        self.classifier = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, bert_inputs, pyg_batch):
        # Move os inputs do BERT para a GPU
        input_ids = bert_inputs['input_ids'].to(device)
        attention_mask = bert_inputs['attention_mask'].to(device)
        
        # Move os inputs da GNN para a GPU
        edge_index = pyg_batch.edge_index.to(device)
        batch_map = pyg_batch.batch.to(device)
        
        # Passa pelo BERT (Fine-tuning)
        # Pegando os embeddings de CADA token
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        token_embeddings = outputs.last_hidden_state 

        # Preparando Features para a GNN
        active_token_embeddings = token_embeddings[attention_mask > 0]
                
        # Passa pela GNN
        h = self.gcn1(active_token_embeddings, edge_index)
        h = F.relu(h)
        h = self.gcn2(h, edge_index)
        h = F.relu(h)
        
        # graph_embedding: (Tamanho do Batch, hidden_dim)
        graph_embedding = self.pool(h, batch_map)
        
        # Classificação
        graph_embedding = self.dropout(graph_embedding)
        logits = self.classifier(graph_embedding)
        
        return logits

# Instanciando o Modelo 
model = HybridGNN_FineTune(
    hidden_dim=HIDDEN_DIM,
    output_dim=num_classes, 
    dropout_prob=0.5
).to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"Modelo {type(model).__name__} instanciado e movido para {device}.")

### Treinamento do Modelo

***Nessa Etapa:***

Iniciamos o ciclo de treinamento do modelo híbrido pelo número de EPOCHS definido. Para cada EPOCH, o código itera sobre todos os lotes de dados, passando os inputs pelo modelo (forward pass). Em seguida, ele calcula o loss, propaga esse erro para trás e ajusta os pesos de todas as camadas (incluindo o BERT e a GNN) usando o otimizador.

In [ ]:
print(f"--- Iniciando Treinamento por {NUM_EPOCHS} épocas ---")
print(f"Otimizador: Adam, LR: {LEARNING_RATE}, Batch Size: {BATCH_SIZE}")

for epoch in range(NUM_EPOCHS):
    
    model.train() 
    total_loss = 0
    start_time = time.time()
    
    for i, batch in enumerate(train_loader):
        bert_inputs = batch['bert_inputs']
        pyg_batch = batch['pyg_batch']
        
        # Pega as labels corretas
        labels = pyg_batch.y.to(device)
        
        # Zera os gradientes
        optimizer.zero_grad()
        
        # Forward pass
        logits = model(bert_inputs, pyg_batch)
        
        # Calcula o loss
        loss = criterion(logits, labels)
        
        # Backward pass 
        loss.backward()
        
        # Otimiza (Ajuste dos pesos)
        optimizer.step()
        
        total_loss += loss.item()
        
        if (i + 1) % 50 == 0:
            print(f"  Época {epoch+1}, Lote {i+1}/{len(train_loader)} | Loss Lote: {loss.item():.4f}")

    # Fim da EPOCH
    end_time = time.time()
    avg_loss = total_loss / len(train_loader)
    epoch_time = end_time - start_time
    
    print(f"\nÉPOCA {epoch+1:02d}/{NUM_EPOCHS} Completa")
    print(f"  Tempo: {epoch_time:.2f}s")
    print(f"  Loss Média (Treino): {avg_loss:.4f}")

print("Treinamento Concluído")

### Avaliação de Desempenho

***Nessa Etapa:***

Executamos o processo de avaliação do modelo treinado. O código coloca o modelo em modo de avaliação. Para cada lote, ele realiza a previsão, armazena os resultados e implementa uma limpeza manual de memória VRAM (torch.cuda.empty_cache()) após cada lote para garantir estabilidade em GPUs com recursos limitados. Ao final do loop, o tempo total é calculado e a acurácia final do modelo no conjunto de teste é exibida

In [ ]:

model.eval() 
all_predictions = []
all_true_labels = []

# Loop de avaliação com limpeza de cache
with torch.no_grad(): 
    
    # Adiciona um timer para ver a velocidade
    start_time = time.time() 
    
    for i, batch in enumerate(test_loader):
        bert_inputs = batch['bert_inputs']
        pyg_batch = batch['pyg_batch']
        
        labels = pyg_batch.y.to(device)
        
        # Faz a previsão
        logits = model(bert_inputs, pyg_batch)
        
        # Pega a classe com maior pontuação
        _, predicted = torch.max(logits, dim=1)
        
        # Salva para cálculo final
        all_predictions.extend(predicted.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

        # Deleta as referências aos tensores da GPU
        del bert_inputs, pyg_batch, labels, logits, predicted
        
        # Força o PyTorch a esvaziar o cache da VRAM
        torch.cuda.empty_cache()

# Calcula a Acurácia Final
end_time = time.time()
accuracy = accuracy_score(all_true_labels, all_predictions)

print(f"\nTempo total de avaliação: {(end_time - start_time):.2f} segundos")
print("\n--- Resultados Finais (com Fine-Tuning) ---")
print(f"Total de Amostras de Teste: {len(all_true_labels)}")
print(f"Acurácia no Teste: {accuracy * 100:.2f}%")

In [ ]:
# caminho para salvar 
dataset = "imdb"  
MODEL_SAVE_PATH = f"{dataset}_hybrid_gnn_finetuned.pth"

# Isso salva apenas os parâmetros aprendidos, não a arquitetura
torch.save(model.state_dict(), MODEL_SAVE_PATH)

print(f"\n--- Modelo Salvo ---")
print(f"Os parâmetros do modelo foram salvos em: {MODEL_SAVE_PATH}")

### Carregamento do Modelo Treinado

Este código tem como objetivo restaurar o modelo treinado: primeiro recria a arquitetura do modelo híbrido e, em seguida, carrega os pesos previamente salvos do disco. O passo final e obrigatório é colocar o modelo no modo de avaliação, preparando-o para ser usado para testes de desempenho ou inferência.

In [ ]:
#  Caminho do modelo salvo
MODEL_SAVE_PATH = "r8_hybrid_gnn_finetuned.pth"
num_classes = 8

# Recria a arquitetura do modelo
print("Instanciando a arquitetura do modelo...")
loaded_model = HybridGNN_FineTune(
    hidden_dim=HIDDEN_DIM,
    output_dim=num_classes,
).to(device)

# Carregar os pesos salvos 
print(f"Carregando pesos de {MODEL_SAVE_PATH}...")
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))

# Desliga o dropout e outras camadas específicas de treino.
loaded_model.eval()

print("\n--- Modelo Carregado e Pronto! ---")
